<a href="https://colab.research.google.com/github/AbhiMalappa/NLP/blob/main/Copy_of_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Data

In [2]:
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [4]:
df = pd.read_csv('training_text_calssifiaction.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'training_text_calssifiaction.csv'

In [ ]:
df.shape
df.sample(5)

(7648, 2)

,string_value,Category
1410,was on hold for 45 mins before an agent came on.,long_holdtime
7083,the electronic portion of the portal was easy.,service_portal
5483,Its frustrating when you call in a ticket and ...,no_resolution
5564,My issue never got resolved now on hold foreve...,no_resolution
6580,"however, navigating the service portal to inpu...",service_portal


In [ ]:
df.Category.value_counts()

,count
Category,
long_holdtime,3222
no_resolution,2197
service_portal,1399
agent_unprofessional,431
Others,399


In [ ]:
df[df.Category.isnull() == True]

,string_value,Category


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train, test = train_test_split(df, test_size = 0.2, shuffle=True, stratify= df.Category, random_state= 777)
print(f'training data shape {train.shape}. Test data shape {test.shape}')

training data shape (6118, 2). Test data shape (1530, 2)


# Vectorization

In [ ]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
#Convert strings to Sparse vector matrix using TFiDF
tfidf = TfidfVectorizer()  # obj
transformed_tfidf = tfidf.fit_transform(train['string_value'])
transformed_tfidf
type(transformed_tfidf) # object details

<6118x4928 sparse matrix of type '<class 'numpy.float64'>'
	with 99523 stored elements in Compressed Sparse Row format>

scipy.sparse._csr.csr_matrix

In [ ]:
hash_vect = HashingVectorizer(n_features=100000)
transformed_hash_vect = hash_vect.fit_transform(train['string_value'])
transformed_hash_vect

<6118x100000 sparse matrix of type '<class 'numpy.float64'>'
	with 99500 stored elements in Compressed Sparse Row format>

In [ ]:
# convert csr_matrix to array
ab = transformed_tfidf.toarray()
ab[0].max()
ab.shape

0.3902245811169529

(6118, 4928)

# Dimensionality Reduction

In [ ]:
from sklearn.decomposition  import TruncatedSVD
from sklearn.decomposition  import PCA

In [ ]:
tsvd = TruncatedSVD(n_components=50) # Desired dimensionality of output data.
type(tsvd)
tsvd

sklearn.decomposition._truncated_svd.TruncatedSVD

TruncatedSVD(n_components=50)

In [ ]:
train_trasformed = tsvd.fit_transform(transformed_tfidf)
train_trasformed.shape
train_trasformed

(6118, 50)

array([[ 0.23429858,  0.14984328, -0.08593963, ...,  0.010694  ,
        -0.06057546, -0.01132903],
       [ 0.15607331, -0.00123569,  0.01185741, ...,  0.03036317,
        -0.04098176,  0.03208501],
       [ 0.25735882,  0.1831351 , -0.0996829 , ...,  0.00892004,
        -0.02178722, -0.0085764 ],
       ...,
       [ 0.20939789, -0.31551903, -0.08506622, ..., -0.03984648,
         0.10074306,  0.04963468],
       [ 0.33565997, -0.15159782, -0.12056716, ..., -0.05766472,
         0.11732092, -0.07340857],
       [ 0.43639335,  0.50557805, -0.48083625, ..., -0.0033106 ,
         0.01357239,  0.01423091]])

In [ ]:
X_train = train_trasformed
y_train = train.Category

X_test = tsvd.fit_transform(tfidf.fit_transform(test['string_value']))
y_test = test.Category

In [ ]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(penalty="l2",random_state=777)
clf.class_weight = "balanced"
clf

LogisticRegression(class_weight='balanced', random_state=777)

In [ ]:
clf.fit(X_train,Y_train)

LogisticRegression(class_weight='balanced', random_state=777)

In [ ]:
predicts = clf.predict(X_test)

In [ ]:
predicts.shape
predicts

(1530,)

array(['no_resolution', 'service_portal', 'agent_unprofessional', ...,
       'long_holdtime', 'long_holdtime', 'agent_unprofessional'],
      dtype=object)

In [ ]:
correct = 0
ab = test.Category.to_numpy()
for i in range(0,len(predicts)):
  if(predicts[i] == ab[i]):
    correct +=1
correct
len(predicts)

1179

1530

In [1]:
from sklearn.metrics import accuracy_score,f1_score
accuracy_score(ab,predicts)
f1_score(ab,predicts)

NameError: name 'ab' is not defined